Data cleaning continuing (part 3)

In [1]:
# Import libraries

import numpy as np
import pandas as pd

In [2]:
# Load datafile
df = pd.read_csv('./dataframes/full_period_4_labelling.csv')
df.head()

,Unnamed: 0,Registration_number,Name,Type,Registered_for_VAT,County,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,...,Deletion_Date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee,Birch_index,emp_growth_rate_CGR,ACE
0,0,10000018,AMSERV AUTO AKTSIASELTS,Äriühing,jah,Harju ( Tallinn ),HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,1134805.23,736604.35,19142710.36,...,1,19142710.36,47.00,1871409.58,1871409.58,0.00,95713.55,0.0,-0.023022,9.0
1,1,10000018,AMSERV AUTO AKTSIASELTS,Äriühing,jah,Harju ( Tallinn ),HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOO...,1375328.87,801558.09,21587224.27,...,1,40729934.63,100.00,2176886.96,4048296.54,0.13,113022.12,0.0,-0.023022,9.0
2,2,10000024,"EESTI RAAMAT, OÜ",Äriühing,jah,Harju maakond,INFO JA SIDE,12126.74,11533.92,51486.75,...,1,51486.75,2.74,23660.66,23660.66,0.00,5720.75,0.0,0.015141,3.0
3,3,10000024,"EESTI RAAMAT, OÜ",Äriühing,jah,Harju maakond,INFO JA SIDE,8484.90,7931.42,59869.39,...,1,111356.14,5.92,16416.32,40076.98,0.16,7483.67,0.0,0.015141,3.0
4,4,10000024,"EESTI RAAMAT, OÜ",Äriühing,jah,Harju maakond,INFO JA SIDE,10468.75,10068.65,55895.57,...,1,167251.71,8.89,20537.40,60614.38,-0.07,6986.95,0.0,0.015141,3.0


In [3]:
# Create second db of company statistics

df_companies = df.groupby('Registration_number').count()
df_companies = df_companies.iloc[:,0:1]
df_companies['Reg_no'] = range(1, len(df_companies) + 1)
df_companies['Reg_no'] = df_companies.index
df_companies.reset_index(drop=True, inplace=True)
df_companies.rename(columns={'Unnamed: 0':'Occurrence'}, inplace=True)
df_companies.head()

,Occurrence,Reg_no
0,2,10000018
1,19,10000024
2,19,10000127
3,19,10000165
4,19,10000219


We decided that if company has less than a year (4 quarters) of usable data it is not enough for the model and therefore we discard these companies

In [4]:
# Discarding companies that have less data

df_companies = df_companies[df_companies['Occurrence']<4]
df_companies.head()

,Occurrence,Reg_no
0,2,10000018
16,3,10000780
66,3,10002997
291,3,10012116
412,3,10018107


In [5]:
df.shape


(1675254, 27)

In [6]:
df = df[~df.Registration_number.isin(df_companies.Reg_no)]
df = df.drop('Unnamed: 0', axis=1)
df.reset_index(drop=True, inplace=True)
df.head()

,Registration_number,Name,Type,Registered_for_VAT,County,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,number_of_employees,...,Deletion_Date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee,Birch_index,emp_growth_rate_CGR,ACE
0,10000024,"EESTI RAAMAT, OÜ",Äriühing,jah,Harju maakond,INFO JA SIDE,12126.74,11533.92,51486.75,9.0,...,1,51486.75,2.74,23660.66,23660.66,0.00,5720.75,0.0,0.015141,3.0
1,10000024,"EESTI RAAMAT, OÜ",Äriühing,jah,Harju maakond,INFO JA SIDE,8484.90,7931.42,59869.39,8.0,...,1,111356.14,5.92,16416.32,40076.98,0.16,7483.67,0.0,0.015141,3.0
2,10000024,"EESTI RAAMAT, OÜ",Äriühing,jah,Harju maakond,INFO JA SIDE,10468.75,10068.65,55895.57,8.0,...,1,167251.71,8.89,20537.40,60614.38,-0.07,6986.95,0.0,0.015141,3.0
3,10000024,"EESTI RAAMAT, OÜ",Äriühing,jah,Harju maakond,INFO JA SIDE,7887.05,7751.34,57334.79,9.0,...,1,224586.50,11.93,15638.39,76252.77,0.03,6370.53,0.0,0.015141,3.0
4,10000024,"EESTI RAAMAT, OÜ",Äriühing,jah,Harju maakond,INFO JA SIDE,9320.26,8757.33,59484.22,10.0,...,1,284070.72,15.09,18077.59,94330.36,0.04,5948.42,0.0,0.015141,3.0


In [7]:
df.shape

(1644634, 26)

In [8]:
# Dropping companies that have not registered for VAT as they are very small companies

df = df[df['Registered_for_VAT']=="jah"]
df = df.drop('Registered_for_VAT', axis=1)
df.shape

(1617211, 25)

In [9]:
df = df.drop('Birch_index', axis=1)
df.reset_index(drop=True, inplace=True)
df.head()

,Registration_number,Name,Type,County,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,number_of_employees,year_quarter,...,Registration_date,Deletion_Date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee,emp_growth_rate_CGR,ACE
0,10000024,"EESTI RAAMAT, OÜ",Äriühing,Harju maakond,INFO JA SIDE,12126.74,11533.92,51486.75,9.0,2017_1,...,1990-07-17,1,51486.75,2.74,23660.66,23660.66,0.00,5720.75,0.015141,3.0
1,10000024,"EESTI RAAMAT, OÜ",Äriühing,Harju maakond,INFO JA SIDE,8484.90,7931.42,59869.39,8.0,2017_2,...,1990-07-17,1,111356.14,5.92,16416.32,40076.98,0.16,7483.67,0.015141,3.0
2,10000024,"EESTI RAAMAT, OÜ",Äriühing,Harju maakond,INFO JA SIDE,10468.75,10068.65,55895.57,8.0,2017_3,...,1990-07-17,1,167251.71,8.89,20537.40,60614.38,-0.07,6986.95,0.015141,3.0
3,10000024,"EESTI RAAMAT, OÜ",Äriühing,Harju maakond,INFO JA SIDE,7887.05,7751.34,57334.79,9.0,2017_4,...,1990-07-17,1,224586.50,11.93,15638.39,76252.77,0.03,6370.53,0.015141,3.0
4,10000024,"EESTI RAAMAT, OÜ",Äriühing,Harju maakond,INFO JA SIDE,9320.26,8757.33,59484.22,10.0,2018_1,...,1990-07-17,1,284070.72,15.09,18077.59,94330.36,0.04,5948.42,0.015141,3.0


In [10]:
# Changing business area as numbered value
df.Business_area_by_EMTAK_system.unique()


array(['INFO JA SIDE',
       'HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOOTORRATASTE REMONT',
       'HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOOTORRATASTE REMONT',
       'TÖÖTLEV TÖÖSTUS', 'KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS',
       'EHITUS', 'PŐLLUMAJANDUS, METSAMAJANDUS JA KALAPÜÜK',
       'PÕLLUMAJANDUS, METSAMAJANDUS JA KALAPÜÜK',
       'KINNISVARAALANE TEGEVUS', 'HALDUS- JA ABITEGEVUSED',
       'VEONDUS JA LAONDUS',
       'VEEVARUSTUS; KANALISATSIOON; JÄÄTME- JA SAASTEKÄITLUS',
       'MAJUTUS JA TOITLUSTUS', 'FINANTS- JA KINDLUSTUSTEGEVUS',
       'KUNST, MEELELAHUTUS JA VABA AEG', 'MUUD TEENINDAVAD TEGEVUSED',
       'MÄETÖÖSTUS',
       'VEEVARUSTUS; KANALISATSIOON, JÄÄTME- JA SAASTEKÄITLUS',
       'TERVISHOID JA SOTSIAALHOOLEKANNE',
       'ELEKTRIENERGIA, GAASI, AURU JA KONDITSIONEERITUD ÕHUGA VARUSTAMINE',
       'HARIDUS',
       'ELEKTRIENERGIA, GAASI, AURU JA KONDITSIONEERITUD ŐHUGA VARUSTAMINE',
       'other',
       'AVALIK HALDUS JA RIIGIKAITSE; KOH

Translating business areas as numeric values as following (TO BE CORRECTED:
1   'INFO JA SIDE',
2   'HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOOTORRATASTE REMONT',
3   'HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOOTORRATASTE REMONT',
4   'TÖÖTLEV TÖÖSTUS'
5   'KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS',
5   'EHITUS', 'PŐLLUMAJANDUS, METSAMAJANDUS JA KALAPÜÜK',
6   'PÕLLUMAJANDUS, METSAMAJANDUS JA KALAPÜÜK',
7   'KINNISVARAALANE TEGEVUS', 'HALDUS- JA ABITEGEVUSED',
8   'VEONDUS JA LAONDUS',
9   'VEEVARUSTUS; KANALISATSIOON; JÄÄTME- JA SAASTEKÄITLUS',
10  'MAJUTUS JA TOITLUSTUS', 'FINANTS- JA KINDLUSTUSTEGEVUS',
11  'KUNST, MEELELAHUTUS JA VABA AEG', 'MUUD TEENINDAVAD TEGEVUSED',
12  'MÄETÖÖSTUS',
13  'VEEVARUSTUS; KANALISATSIOON, JÄÄTME- JA SAASTEKÄITLUS',
14  'TERVISHOID JA SOTSIAALHOOLEKANNE',
15  'ELEKTRIENERGIA, GAASI, AURU JA KONDITSIONEERITUD ÕHUGA VARUSTAMINE',
16  'HARIDUS',
17  'ELEKTRIENERGIA, GAASI, AURU JA KONDITSIONEERITUD ŐHUGA VARUSTAMINE',
18  'other',
19  'AVALIK HALDUS JA RIIGIKAITSE; KOHUSTUSLIK SOTSIAALKINDLUSTUS',
20  'KODUMAJAPIDAMISTE KUI TÖÖANDJATE TEGEVUS; KODUMAJAPIDAMISTE OMA TARBEKS MÕELDUD ERISTAMATA KAUPAD',
21  ' AVALIK HALDUS JA RIIGIKAITSE; KOHUSTUSLIK SOTSIAALKINDLUSTUS']

In [11]:
dict = {"INFO JA SIDE" : 1,\
        "HULGI- JA JAEKAUBANDUS; MOOTORSŐIDUKITE JA MOOTORRATASTE REMONT" : 2,\
        "HULGI- JA JAEKAUBANDUS; MOOTORSÕIDUKITE JA MOOTORRATASTE REMONT" : 3,\
        "TÖÖTLEV TÖÖSTUS" : 4,\
        "KUTSE-, TEADUS- JA TEHNIKAALANE TEGEVUS" : 5,\
        "EHITUS" : 6,\
        "PŐLLUMAJANDUS, METSAMAJANDUS JA KALAPÜÜK" : 7,\
        "PÕLLUMAJANDUS, METSAMAJANDUS JA KALAPÜÜK" : 8,\
        "KINNISVARAALANE TEGEVUS" : 9,\
        "HALDUS- JA ABITEGEVUSED" : 10,\
        "VEONDUS JA LAONDUS" : 11,\
        "VEEVARUSTUS; KANALISATSIOON; JÄÄTME- JA SAASTEKÄITLUS" : 12,\
        "MAJUTUS JA TOITLUSTUS" : 13,\
        "FINANTS- JA KINDLUSTUSTEGEVUS" : 14,\
        "KUNST, MEELELAHUTUS JA VABA AEG" : 15,\
        "MUUD TEENINDAVAD TEGEVUSED" : 16,\
        "MÄETÖÖSTUS" : 17,\
        "VEEVARUSTUS; KANALISATSIOON, JÄÄTME- JA SAASTEKÄITLUS" : 18,\
        "TERVISHOID JA SOTSIAALHOOLEKANNE" : 19,\
        "ELEKTRIENERGIA, GAASI, AURU JA KONDITSIONEERITUD ÕHUGA VARUSTAMINE" : 20,\
        "HARIDUS" : 21,\
        "ELEKTRIENERGIA, GAASI, AURU JA KONDITSIONEERITUD ŐHUGA VARUSTAMINE" : 22,\
        "other" : 23,\
        "AVALIK HALDUS JA RIIGIKAITSE; KOHUSTUSLIK SOTSIAALKINDLUSTUS" : 24,\
        "KODUMAJAPIDAMISTE KUI TÖÖANDJATE TEGEVUS; KODUMAJAPIDAMISTE OMA TARBEKS MÕELDUD ERISTAMATA KAUPAD" : 25,\
        " AVALIK HALDUS JA RIIGIKAITSE; KOHUSTUSLIK SOTSIAALKINDLUSTUS" : 26}

In [12]:
df = df.replace({"Business_area_by_EMTAK_system": dict})
df.head()

,Registration_number,Name,Type,County,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,number_of_employees,year_quarter,...,Registration_date,Deletion_Date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee,emp_growth_rate_CGR,ACE
0,10000024,"EESTI RAAMAT, OÜ",Äriühing,Harju maakond,1,12126.74,11533.92,51486.75,9.0,2017_1,...,1990-07-17,1,51486.75,2.74,23660.66,23660.66,0.00,5720.75,0.015141,3.0
1,10000024,"EESTI RAAMAT, OÜ",Äriühing,Harju maakond,1,8484.90,7931.42,59869.39,8.0,2017_2,...,1990-07-17,1,111356.14,5.92,16416.32,40076.98,0.16,7483.67,0.015141,3.0
2,10000024,"EESTI RAAMAT, OÜ",Äriühing,Harju maakond,1,10468.75,10068.65,55895.57,8.0,2017_3,...,1990-07-17,1,167251.71,8.89,20537.40,60614.38,-0.07,6986.95,0.015141,3.0
3,10000024,"EESTI RAAMAT, OÜ",Äriühing,Harju maakond,1,7887.05,7751.34,57334.79,9.0,2017_4,...,1990-07-17,1,224586.50,11.93,15638.39,76252.77,0.03,6370.53,0.015141,3.0
4,10000024,"EESTI RAAMAT, OÜ",Äriühing,Harju maakond,1,9320.26,8757.33,59484.22,10.0,2018_1,...,1990-07-17,1,284070.72,15.09,18077.59,94330.36,0.04,5948.42,0.015141,3.0


In [13]:
# Drop all textual columns

df = df.drop('Name', axis=1)
df = df.drop('Type', axis=1)
df = df.drop('County', axis=1)
df = df.drop('EMTAK_by_letter', axis=1)
df = df.drop('Institutional_classification', axis=1)


In [14]:
df.shape


(1617211, 19)

In [15]:
df.head()

,Registration_number,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,number_of_employees,year_quarter,EMTAK_by_number,Client_ID_by_Eesti_Pank,Registration_date,Deletion_Date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee,emp_growth_rate_CGR,ACE
0,10000024,1,12126.74,11533.92,51486.75,9.0,2017_1,5811.0,8.0,1990-07-17,1,51486.75,2.74,23660.66,23660.66,0.00,5720.75,0.015141,3.0
1,10000024,1,8484.90,7931.42,59869.39,8.0,2017_2,5811.0,8.0,1990-07-17,1,111356.14,5.92,16416.32,40076.98,0.16,7483.67,0.015141,3.0
2,10000024,1,10468.75,10068.65,55895.57,8.0,2017_3,5811.0,8.0,1990-07-17,1,167251.71,8.89,20537.40,60614.38,-0.07,6986.95,0.015141,3.0
3,10000024,1,7887.05,7751.34,57334.79,9.0,2017_4,5811.0,8.0,1990-07-17,1,224586.50,11.93,15638.39,76252.77,0.03,6370.53,0.015141,3.0
4,10000024,1,9320.26,8757.33,59484.22,10.0,2018_1,5811.0,8.0,1990-07-17,1,284070.72,15.09,18077.59,94330.36,0.04,5948.42,0.015141,3.0


In [17]:
# Saving data to file
df.to_csv('././dataframes/full_period_unlabeled.csv')


In [18]:

# Filter out data that is before 1st quarter of 2020

df_before = df.copy(deep=True)

In [19]:
df_before = df_before.loc[df_before.apply(lambda x: x.year_quarter in ['2017_1', '2017_2', '2017_3', '2017_4', '2018_1', '2018_2', '2018_3', '2018_4', '2019_1', '2019_2', '2019_3', '2019_4', '2020_1'], axis=1)]

In [20]:
df_before.head()

,Registration_number,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,number_of_employees,year_quarter,EMTAK_by_number,Client_ID_by_Eesti_Pank,Registration_date,Deletion_Date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee,emp_growth_rate_CGR,ACE
0,10000024,1,12126.74,11533.92,51486.75,9.0,2017_1,5811.0,8.0,1990-07-17,1,51486.75,2.74,23660.66,23660.66,0.00,5720.75,0.015141,3.0
1,10000024,1,8484.90,7931.42,59869.39,8.0,2017_2,5811.0,8.0,1990-07-17,1,111356.14,5.92,16416.32,40076.98,0.16,7483.67,0.015141,3.0
2,10000024,1,10468.75,10068.65,55895.57,8.0,2017_3,5811.0,8.0,1990-07-17,1,167251.71,8.89,20537.40,60614.38,-0.07,6986.95,0.015141,3.0
3,10000024,1,7887.05,7751.34,57334.79,9.0,2017_4,5811.0,8.0,1990-07-17,1,224586.50,11.93,15638.39,76252.77,0.03,6370.53,0.015141,3.0
4,10000024,1,9320.26,8757.33,59484.22,10.0,2018_1,5811.0,8.0,1990-07-17,1,284070.72,15.09,18077.59,94330.36,0.04,5948.42,0.015141,3.0


In [21]:
df_before.shape

(1069895, 19)

In [22]:
# Save acquired data into a csv

df_before.to_csv( "before_covid_unlabelled.csv", index=False)

In [23]:
# Filter out data that is since 2nd quarter of 2020

df_during = df.copy(deep=True)

In [24]:
df_during = df_during.loc[df_during.apply(lambda x: x.year_quarter in ['2020_2', '2020_3', '2020_4', '2021_1', '2021_2', '2021_3'], axis=1)]


In [25]:
df_during.head()

,Registration_number,Business_area_by_EMTAK_system,All_taxes,Labor_taxes,Turnover,number_of_employees,year_quarter,EMTAK_by_number,Client_ID_by_Eesti_Pank,Registration_date,Deletion_Date,cum_sum_turnover,cum_perc_turnover,combined_taxes,cum_sum_combined_taxes,revenue_growth,profit_per_employee,emp_growth_rate_CGR,ACE
13,10000024,1,18474.78,10983.73,137159.52,9.0,2020_2,5811.0,8.0,1990-07-17,1,1179457.60,62.67,29458.51,350586.87,0.06,15239.95,0.015141,3.0
14,10000024,1,18969.94,14565.42,107660.58,10.0,2020_3,5811.0,8.0,1990-07-17,1,1287118.18,68.39,33535.36,384122.23,-0.22,10766.06,0.015141,3.0
15,10000024,1,20981.29,15506.56,147818.28,10.0,2020_4,5811.0,8.0,1990-07-17,1,1434936.46,76.25,36487.85,420610.08,0.37,14781.83,0.015141,3.0
16,10000024,1,22353.85,18034.91,140346.67,11.0,2021_1,5811.0,8.0,1990-07-17,1,1575283.13,83.71,40388.76,460998.84,-0.05,12758.79,0.015141,3.0
17,10000024,1,19685.75,18454.83,162231.81,12.0,2021_2,5811.0,8.0,1990-07-17,1,1737514.94,92.33,38140.58,499139.42,0.16,13519.32,0.015141,3.0


In [26]:
df_during.shape

(547316, 19)

In [27]:
# Save acquired data into a csv

df_during.to_csv( "during_covid_unlabelled.csv", index=False)